## Heads up

This was utilizing a follow along lesson using the previous generation tensorflow libaries whil I at the current time of writing this installed 3 month old TensorFlow 2.0 beta. This follows the famous irids flower dataset and I attempted to implement both practices with fruitless results. Im leaving this for posterity purposes for now in case i can go back and utilize train and test traditional practices more effectively here later. Otherwise check files 18-05a instead. 


* TF has an Estimator object we can use to quickly create models with needing to manually define the graph as we did before. 

* Estimator steps. 

    * Read in Data (normalize if necessary)
    * Train/Test split the data 
    * Create Estimator Feature Columns
    * Create Input Estimator Function
    * Train Estimator Model
    * Predict with new Test Input Function

In [1]:
import tensorflow as tf
import pandas as pd


In [2]:
df = pd.read_csv('iris.csv')

In [4]:
df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
sepal length (cm)    150 non-null float64
sepal width (cm)     150 non-null float64
petal length (cm)    150 non-null float64
petal width (cm)     150 non-null float64
target               150 non-null float64
dtypes: float64(5)
memory usage: 5.9 KB


In [6]:
df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.054000,3.758667,1.198667,1.000000
std,0.828066,0.433594,1.764420,0.763161,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [7]:
df.columns

Index(['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)',
       'petal width (cm)', 'target'],
      dtype='object')

In [8]:
df.columns = ['sepal_length', 'sepal_width', 'petal_length',
       'petal_width', 'target']

In [9]:
df.head()

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0.0
1,4.9,3.0,1.4,0.2,0.0
2,4.7,3.2,1.3,0.2,0.0
3,4.6,3.1,1.5,0.2,0.0
4,5.0,3.6,1.4,0.2,0.0


In [53]:
df['target'] = df['target'].apply(int)

In [54]:
df.head(1)

,sepal_length,sepal_width,petal_length,petal_width,target
0,5.1,3.5,1.4,0.2,0


In [55]:
y =df['target']

In [56]:
X = df.drop('target',axis=1)

In [57]:
from sklearn.model_selection import train_test_split

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

In [15]:
import tensorflow as tf

In [59]:
X.columns

Index(['sepal_length', 'sepal_width', 'petal_length', 'petal_width'], dtype='object')

In [60]:
# okay either it follows exactly like it does in the video or we get
# creative and take this one step at a time


feat_cols = []

for col in X.columns:
    feat_cols.append(tf.feature_column.numeric_column(col))

In [61]:
feat_cols

[NumericColumn(key='sepal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='sepal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_length', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None),
 NumericColumn(key='petal_width', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]

In [62]:
classifier = tf.estimator.DNNClassifier(feature_columns=feat_cols,hidden_units=[10,20,10], n_classes=3)

W0731 15:10:48.355447 10516 estimator.py:1811] Using temporary folder as model directory: C:\Users\Corvus\AppData\Local\Temp\tmpjucbdo99


In [80]:
def input_fn(x,y, training=True, batch_size=256):
    """An input function for training or evaluating"""
    # Convert the inputs to a Dataset.
    dataset = tf.data.Dataset.from_tensor_slices((dict(x), y))

    # Shuffle and repeat if you are in training mode.
    if training:
        dataset = dataset.shuffle(1000).repeat()
    
    return dataset.batch(batch_size)

In [81]:
classifier = tf.estimator.DNNClassifier(hidden_units= [10,20,10],n_classes=3,feature_columns=feat_cols)

W0731 15:21:09.153955 10516 estimator.py:1811] Using temporary folder as model directory: C:\Users\Corvus\AppData\Local\Temp\tmpshm40617


In [82]:
classifier.train(
    input_fn=lambda: input_fn(X_train, y_train, training=True),
    steps=5000)

In [102]:
eval_result =classifier.evaluate(input_fn=lambda: input_fn(X_train, y_train, training=False))
print('\nTest set accuracy: {accuracy:0.3f}\n'.format(**eval_result))


Test set accuracy: 0.667



In [107]:
def input_fn(X_train, batch_size=256):
    """An input function for prediction."""
    # Convert the inputs to a Dataset without labels.
    return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)

predictions = classifier.predict(input_fn=lambda: input_fn(eval_result))

TypeError: input_fn() missing 1 required positional argument: 'y'